In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imripy.halo
from imripy import merger_system as ms
import time

## Compare the analytic mass function of the halo profiles with the numeric integration

In [ ]:
def CompareMassFunction(halo, r_grid, ax):
    tic = time.perf_counter()
    m_ana = halo.mass(r_grid)  # Here we call the analytic expression
    toc = time.perf_counter()
    t_ana = toc - tic

    tic = time.perf_counter()
    m_num = imripy.halo.MatterHalo.mass(halo, r_grid) # Here we call the numerical odeint integration in the base class
    #m_num = [imripy.halo.MatterHalo.mass(halo, r) for r in r_grid]  # Here we call the numerical quad integration in the base class
    toc = time.perf_counter()
    t_num = toc - tic

    print("Comparing implementation for " + str(halo))
    print("elapsed time in analytic case: ", t_ana)
    print("elapsed time in numeric case:", t_num)
    print("The average relative error is ", np.average(np.abs(m_num/m_ana - 1.)))
    print("The maximal relative error is ", np.max(np.abs(m_num/m_ana - 1.)))

    l, = ax.loglog(r_grid, m_ana, alpha=0.5, label=str(halo) + ',analytic')
    ax.loglog(r_grid, m_num, color=l.get_c(), linestyle='--', label=str(halo) + ',numeric')

In [ ]:
n = 1000
r_grid = np.geomspace(1e-5, 1e5, n)

# Test numerical and analytical mass functions
plt.figure(figsize=(16, 10)); ax = plt.gca()
CompareMassFunction(imripy.halo.ConstHalo(1.), r_grid, ax)
CompareMassFunction(imripy.halo.NFW(1., 1e2), r_grid, ax)
CompareMassFunction(imripy.halo.SpikedNFW(1., 1e3, 1e-2, 7./3.), r_grid, ax)
CompareMassFunction(imripy.halo.Spike(1., 1e-2, 7./3.), r_grid, ax)
CompareMassFunction(imripy.halo.Hernquist(1., 1e0), r_grid, ax)
ax.set_xlabel("r")
ax.set_ylabel("m")
plt.grid(); plt.legend();
plt.show()

## Compare the results from the Eddington inversion procedure with the analytic equations

In [ ]:
from scipy.integrate import quad, odeint
from scipy.interpolate import interp1d

In [ ]:
def TestEddingtonInversion(halo, r_grid, ax_r, ax_eps, extPotential=None, f_ana=None, pot_ana=None):
    if extPotential is None:  # Invert density function in its own potential
        integrand = lambda r, m: halo.mass(np.abs(r))/r**2
        Phi_inf = quad(integrand, r_grid[-1], np.inf, args=(0.), limit=200)[0]
        Phi_inf = np.clip(Phi_inf, 1e-50, None)
        extPotential =  odeint(integrand, Phi_inf, -r_grid[::-1], tfirst=True, atol=1e-10, rtol=1e-10)[::-1,0]
        extPotential = interp1d(r_grid, extPotential, kind='cubic', bounds_error=False, fill_value=(0.,0.))

    if not pot_ana is None:
        l, = ax_r.loglog(r_grid, extPotential(r_grid), linestyle='--', label=str(halo) + ' $\Phi$, recovered')
        ax_r.loglog(r_grid, pot_ana(r_grid), color=l.get_c(), alpha=0.5, label=str(halo) + ' $\Phi$, analytic')

    Eps_grid = np.geomspace(extPotential(r_grid[-1]), extPotential(r_grid[0]), 500)
    haloRec = imripy.halo.DynamicSS.FromStatic(Eps_grid, halo, extPotential)   # Here the Eddington inversion is called
    
    l, = ax_r.loglog(r_grid, halo.density(r_grid), alpha=0.5, label=str(halo) + ',static')  # Plot for comparison
    ax_r.loglog(r_grid, haloRec.density(r_grid), color=l.get_c(), linestyle='--', label=str(halo) + ',recovered')
    ax_eps.loglog(Eps_grid, haloRec.f_grid, color=l.get_c(), linestyle='--', label=str(halo) +',recovered')
    if not f_ana is None:
        ax_eps.loglog(Eps_grid, f_ana(Eps_grid), color=l.get_c(), alpha = 0.5 , label=str(halo) + ',analytic')
    return haloRec

In [ ]:
fig, (ax_r, ax_eps) = plt.subplots(2, 1, figsize=(20,20));
n = 1000
r_grid = np.geomspace(1e-5, 1e5, n)

# Test Eddington inversion for Spike profile with known analytic equations
Spike = imripy.halo.Spike(1., 1e-2, 7./3.)
extPotential = lambda r : 1./r
from scipy.special import gamma
f_ana = lambda E : Spike.rho_spike * Spike.alpha*(Spike.alpha-1.)/(2.*np.pi)**(3./2.) * (Spike.r_spike/1.)**Spike.alpha * gamma(Spike.alpha-1.)/gamma(Spike.alpha-1./2.) * E**(Spike.alpha-3./2.)
TestEddingtonInversion( Spike , r_grid, ax_r, ax_eps, extPotential, f_ana)

# Test Eddington inversion for Hernquist profile with known analytic equations
Hern = imripy.halo.Hernquist(1., 1e1)
def f_ana_Hern(Eps):
    M = 2.*np.pi * Hern.rho_s * Hern.r_s**3
    E = Eps*Hern.r_s/M
    return (M * Hern.r_s)**(-3./2.)/np.sqrt(2)/(2.*np.pi)**3 * np.sqrt(E)/(1-E)**2  *( (1.-2*E)*(8.*E**2 - 8.*E - 3.) + 3.*np.arcsin(np.sqrt(E)) / np.sqrt(E*(1-E))  )
def pot_ana_Hern(r):
    M = 2.*np.pi * Hern.rho_s * Hern.r_s**3
    return M/(r + Hern.r_s)

TestEddingtonInversion( Hern , r_grid, ax_r, ax_eps, extPotential=None, f_ana=f_ana_Hern, pot_ana=pot_ana_Hern)

ax_eps.grid(); ax_eps.legend(); ax_eps.set_xlabel(r"$\varepsilon$"); ax_eps.set_ylabel("f")
ax_r.grid(); ax_r.legend(); ax_r.set_xlabel("r")
plt.show()

### Compare Spike parametrizations

In [ ]:
m1 = 1e3 * ms.solar_mass_to_pc
alpha = 7./3.
rho_spike = 226*ms.solar_mass_to_pc
r_spike = ( (3-alpha)* 0.2**(3-alpha) * m1 / 2. / np.pi / rho_spike)**(1./3)
s1 = imripy.halo.Spike(rho_spike, r_spike, alpha)
rho_6 = 5.448*1e15 * ms.solar_mass_to_pc
s2 = imripy.halo.Spike.FromRho6(rho_6, m1, alpha)
print(s1.rho_spike, s1.r_spike, s1.alpha)
print(s2.rho_spike, s2.r_spike, s2.alpha)